In [5]:
import numpy as np
import pandas as pd

class FS():
    def __init__(self,X, y, labeled):
        self.X = X
        self.nDim = X.shape[1]
        self.labels = np.unique(y)
        self.nClass = len(np.unique(y))
        self.y = y
        self.labeled = labeled
        self.fs = self.attribute_weight()

    def attribute_weight(self):
        # ------------Initialize some things------------
        total_Mean = np.mean(self.X[self.labeled], axis=0)
        class_Num = np.zeros(self.nClass)
        class_Mean = np.zeros((self.nClass, self.nDim))
        class_std = np.zeros((self.nClass, self.nDim))
        fisher_score_list = np.zeros(self.nDim)

        # --------calculate intermediate components---------
        labeled_y = self.y[self.labeled]
        labeled_idx = np.array(self.labeled)
        for i, lab in enumerate(self.labels):
            ids = np.where(labeled_y == lab)[0]
            lab_ids = labeled_idx[ids]  # 类别为lab的已标记样本索引
            class_Num[i] = len(lab_ids)
            class_Mean[i] = np.mean(self.X[lab_ids], axis=0)
            class_std[i] = np.std(self.X[lab_ids], axis=0)

        # --utlize the intermediate components to compute the FS------
        for j in range(self.nDim):
            Sb_j = 0.0
            Sw_j = 0.0
            for i in range(self.nClass):
                Sb_j += class_Num[i] * (class_Mean[i,j] - total_Mean[j]) ** 2
                Sw_j += class_Num[i] * class_std[i,j] ** 2
            fisher_score_list[j] = Sb_j / Sw_j
        print("FS::",fisher_score_list)
        fisher_score_list = np.where(np.isnan(fisher_score_list), 0, fisher_score_list)
        print("FS::",fisher_score_list)

        # -----softmax based normalize FS range to [0,1]------
        Norm_fs = np.exp(fisher_score_list)
        Norm_fs_sum = sum(Norm_fs)
        Norm_fs = Norm_fs / Norm_fs_sum

        return Norm_fs

df=pd.read_csv('data/lym/lymphography.csv')
feature_columns = [ col for col in df.columns if col not in ['id','target']]
Y = df["target"]
X = df[feature_columns]

# Compute the feature importance using Fisher score
fs_model = FS(X=X.values, y=Y.values, labeled=df['id'].values-1)
feature_importance = fs_model.fs

# Print the feature importance values
print("Feature importance:", feature_importance)

top_indexes = np.argsort(feature_importance)[::-1][:12]
top_features = [feature_columns[index] for index in top_indexes]
print(top_features)

FS:: [0.07386648 0.29526465 0.05010312 0.14920407 0.11719727 0.03085578
 0.67167948 0.20228919 4.31103167 0.29909009 0.19221567 0.07964696
 0.15979881 0.12458497 0.20534982 0.09023947 0.10467788 0.44068065]
FS:: [0.07386648 0.29526465 0.05010312 0.14920407 0.11719727 0.03085578
 0.67167948 0.20228919 4.31103167 0.29909009 0.19221567 0.07964696
 0.15979881 0.12458497 0.20534982 0.09023947 0.10467788 0.44068065]
Feature importance: [0.01128305 0.01407921 0.01101808 0.01216592 0.0117827  0.01080804
 0.02051413 0.0128292  0.78091503 0.01413317 0.01270061 0.01134846
 0.0122955  0.01187006 0.01286853 0.0114693  0.0116361  0.0162829 ]
['f10', 'f8', 'f19', 'f11', 'f3', 'f16', 'f9', 'f12', 'f14', 'f5', 'f15', 'f6']
FS:: [0.07386648 0.29526465 0.05010312 0.14920407 0.11719727 0.03085578
 0.67167948 0.20228919 4.31103167 0.29909009 0.19221567 0.07964696
 0.15979881 0.12458497 0.20534982 0.09023947 0.10467788 0.44068065]
FS:: [0.07386648 0.29526465 0.05010312 0.14920407 0.11719727 0.03085578
 0.67

In [6]:
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold

df=pd.read_csv('data/lym/lymphography.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'target']]
Y = df["target"]
X = df[top_features]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=500)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

0.7895238095238095


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


0.7895238095238095


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


In [7]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df=pd.read_csv('data/lym/lymphography.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
X = df[top_features]
Y = df['target']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.7628571428571428
0.7485714285714284


In [8]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df=pd.read_csv('data/lym/lymphography.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
Y=df["target"]
X=df[top_features]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.7752380952380953
0.7752380952380953
